# Creating Pandapower Networks

This Tutorial will introduce the user into the pandapower datastructure and how to create networks through the pandapower API. The following minimal example contains the most commont elements that are supported by the Pandapower format. For an example that contains all pandapower elements (3 winding transformers, ward equivalents, impedances), see the [advanced tutorial](create_advanced.ipynb) for network creation.

<img src="pics/example_network_simple.png">

The datastructure of the pandapower framework is based on the python library pandas. A pandapower network consist of a separate element table for each element type that is used in the network. Each element table consists of a column for each parameter and a row for each element. By executing the follwing code cells you generate the various element tables.  You can find detailed descriptions about each parameter in the pandapower documentation under bulletpoint "Datastructures and Elements".

### Empty Network

First, we import pandapower an create an empty network:

In [2]:
import pandapower as pp #import pandapower

net = pp.create_empty_network() #create an empty network

### Buses

<img src="pics/example_network_simple_buses.png">

We now create the three high voltage (vn_kv=110.) and six medium voltage (vn_kv=20.) buses.

In [3]:
bus1 = pp.create_bus(net, name="HV Busbar", vn_kv=110, type="b")
bus2 = pp.create_bus(net, name="HV Busbar 2", vn_kv=110, type="b")
bus3 = pp.create_bus(net, name="HV Transformer Bus", vn_kv=110, type="n")
bus4 = pp.create_bus(net, name="MV Transformer Bus", vn_kv=20, type="n")
bus5 = pp.create_bus(net, name="MV Main Bus", vn_kv=20, type="b")
bus6 = pp.create_bus(net, name="MV Bus 1", vn_kv=20, type="b")
bus7 = pp.create_bus(net, name="MV Bus 2", vn_kv=20, type="b")

Bus 3 and bus 4 are classified as nodes (type="n"), all other bus types are declared as busbars (type="b"):

In [4]:
net.bus # show bus table

,name,vn_kv,type,zone,in_service
0,HV Busbar,110.0,b,None,True
1,HV Busbar 2,110.0,b,None,True
2,HV Transformer Bus,110.0,n,None,True
3,MV Transformer Bus,20.0,n,None,True
4,MV Main Bus,20.0,b,None,True
5,MV Bus 1,20.0,b,None,True
6,MV Bus 2,20.0,b,None,True


All create functions return the pandapower index of the element that was created, for example the variable bus6 is now equal to the index of the bus with the name "MV Station 2" (which is 5):

In [5]:
bus6

5

We use these variables for creating bus and branch elements in the following.

### External Grid

<img src="pics/example_network_simple_ext_grid.png">

We now create an external grid connection that serves as slack node for the power flow calculation. The voltage of the external grid is set to a magnitude of 1.02 per unit and 50 degrees voltage angle:

In [6]:
pp.create_ext_grid(net, bus1, vm_pu=1.02, va_degree=50) # Create an external grid connection

net.ext_grid #show external grid table

,name,bus,vm_pu,va_degree,in_service
0,None,0,1.02,50.0,True


### Transformer 

<img src="pics/example_network_simple_trafo.png">

The transformer connects the high-voltage with the medium-voltage side of the grid. The high-voltage bus of the transformer is connected to Bus3 and on the medium-voltage side the transformer is linked to Bus4. We select the standard type "25 MVA 110/20 kV" from the pandapower basic standard type library:

In [7]:
trafo1 = pp.create_transformer(net, bus3, bus4, name="110kV/20kV transformer", std_type="25 MVA 110/20 kV")

 The detailled transformer parameters, such as short circuit voltages, rated power or iron losses are automatically loaded from the standard type library (see [standard type library tutorial](std_types.ipynb)) and stored in the transformer table:

In [8]:
net.trafo #show transformer table

,name,std_type,hv_bus,lv_bus,sn_kva,vn_hv_kv,vn_lv_kv,vsc_percent,vscr_percent,pfe_kw,i0_percent,shift_degree,tp_side,tp_mid,tp_min,tp_max,tp_st_percent,tp_pos,in_service
0,110kV/20kV transformer,25 MVA 110/20 kV,2,3,25000.0,110.0,20.0,11.2,0.282,29.0,0.071,150.0,hv,0,-9,9,1.5,0,True


### Lines

The network includes three medium voltage lines and one high voltage line. The bus connections and line lengths are defined in the network diagram:

<img src="pics/example_network_simple_lines.png">

The line parameters are once again taken from the standard type library (see [standard type library tutorial](std_types.ipynb)). We use different line lengths and standard types for each line:

In [9]:
line1 = pp.create_line(net, bus1, bus2, length_km=10, std_type="N2XS(FL)2Y 1x300 RM/35 64/110 kV",  name="Line 1")
line2 = pp.create_line(net, bus5, bus6, length_km=2.0, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 2")
line3 = pp.create_line(net, bus6, bus7, length_km=3.5, std_type="48-AL1/8-ST1A 20.0", name="Line 3")
line4 = pp.create_line(net, bus7, bus5, length_km=2.5, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 4")

The full line table looks like this:

In [10]:
net.line # show line table

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,imax_ka,df,parallel,type,in_service
0,Line 1,N2XS(FL)2Y 1x300 RM/35 64/110 kV,0,1,10.0,0.0600,0.144,144.0,0.588,1.0,1,cs,True
1,Line 2,NA2XS2Y 1x240 RM/25 12/20 kV,4,5,2.0,0.1220,0.112,304.0,0.421,1.0,1,cs,True
2,Line 3,48-AL1/8-ST1A 20.0,5,6,3.5,0.5939,0.372,9.5,0.210,1.0,1,ol,True
3,Line 4,NA2XS2Y 1x240 RM/25 12/20 kV,6,4,2.5,0.1220,0.112,304.0,0.421,1.0,1,cs,True


### Switches

There are two circuit breakers on the high- and low voltage side of the transformer which connect two buses (Bus2-Bus3 and Bus4-Bus5). These bus-bus switches can be defined with et="b". 
<img src="pics/example_network_simple_switches.png">

In [11]:
sw1 = pp.create_switch(net, bus2, bus3, et="b", type="CB", closed=True)
sw2 = pp.create_switch(net, bus4, bus5, et="b", type="CB", closed=True)

Furthermore, we equip all bus/line connections in the medium voltage level with load break switches ("LBS") as shown in the network diagram. Bus/Line switches are defined with et="l":

In [12]:
sw3 = pp.create_switch(net, bus5, line2, et="l", type="LBS", closed=True)
sw4 = pp.create_switch(net, bus6, line2, et="l", type="LBS", closed=True)
sw5 = pp.create_switch(net, bus6, line3, et="l", type="LBS", closed=True)
sw6 = pp.create_switch(net, bus7, line3, et="l", type="LBS", closed=False)
sw7 = pp.create_switch(net, bus7, line4, et="l", type="LBS", closed=True)
sw8 = pp.create_switch(net, bus5, line4, et="l", type="LBS", closed=True)

The switch table now shows all switches. The bus colum contains the index of the bus the switch is connected to. For bus switches (et="b"), the element column contains the index of the second bus the switch connects to. For line switches (et="l"), the element column contains the index of the line the switch connects to. All switches are closed.

In [13]:
net.switch # show switch table

,bus,element,et,type,closed,name
0,1,2,b,CB,True,None
1,3,4,b,CB,True,None
2,4,1,l,LBS,True,None
3,5,1,l,LBS,True,None
4,5,2,l,LBS,True,None
5,6,2,l,LBS,False,None
6,6,3,l,LBS,True,None
7,4,3,l,LBS,True,None


### Load

<img src="pics/example_network_simple_load.png">

The load element is used to model constant active and reactive power consumption. We create a 2 MW / 4 MVar load with a scaling factor of 0.6:

In [14]:
pp.create_load(net, bus7, p_kw=2000, q_kvar=4000, scaling=0.6, name="load")

net.load

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type
0,load,6,2000.0,4000.0,NaN,0.6,True,None


### Static Generator

<img src="pics/example_network_simple_sgen.png">

The static generator element is used to model constant active and reactive power generation. Since the signing system used in pandapower is always from a consumers point of view, the active power has to be negative to model generation. We create a static generator with 2 MW generation and 500

In [15]:
pp.create_sgen(net, bus7, p_kw=-2000, q_kvar=500, name="static generator")

net.sgen

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type
0,static generator,6,-2000.0,500.0,NaN,1.0,True,None


### Voltage controlled Generator

<img src="pics/example_network_simple_gen.png">

The generator element is used to model voltage controlled active power generation. We define it with a active power generation (negative) and a voltage set point:

In [16]:
pp.create_gen(net, bus6, p_kw=-6000, max_q_kvar=3000, min_q_kvar=-3000, vm_pu=1.03, name="generator") 

net.gen

,name,bus,p_kw,vm_pu,sn_kva,min_q_kvar,max_q_kvar,scaling,in_service,type
0,generator,5,-6000.0,1.03,NaN,-3000.0,3000.0,1.0,True,None


### Shunt


<img src="pics/example_network_simple_shunt.png">

The shunt is defined by its active and reactive power consumption at rated voltage. Once again, the signing system is from a consumers point of view. We want to model a a capacitator bank, and therefore have to assign a negative reactive power to the shunt:

In [17]:
pp.create_shunt(net, bus3, q_kvar=-960, p_kw=0, name='Shunt')

net.shunt

,bus,name,q_kvar,p_kw,in_service
0,2,Shunt,-960.0,0.0,1


If you want to learn how to create more complicated networks, continue with the [advanced create tutorial](create_advanced.ipynb). If you want to learn about how to run a loadflow, continue with the [power flow tutorial](powerflow.ipynb). 